<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Preliminaries" data-toc-modified-id="Preliminaries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Preliminaries</a></span><ul class="toc-item"><li><span><a href="#Data_SRC" data-toc-modified-id="Data_SRC-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Data_SRC</a></span></li><li><span><a href="#Load-libraries" data-toc-modified-id="Load-libraries-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Load libraries</a></span></li></ul></li><li><span><a href="#Demo-of-process_mpd" data-toc-modified-id="Demo-of-process_mpd-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Demo of process_mpd</a></span><ul class="toc-item"><li><span><a href="#counting-the-number-of-playlist" data-toc-modified-id="counting-the-number-of-playlist-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>counting the number of playlist</a></span></li><li><span><a href="#get-unique-tracks-in-all-playlist" data-toc-modified-id="get-unique-tracks-in-all-playlist-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>get unique tracks in all playlist</a></span></li><li><span><a href="#create-random-subset-of-1%-of-all-playlists" data-toc-modified-id="create-random-subset-of-1%-of-all-playlists-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>create random subset of 1% of all playlists</a></span></li></ul></li></ul></div>

# Preliminaries
## Data_SRC

In [33]:
#location of the dataset
#DATA_SRC='/home/beangoben/ml_data/mpd.v1/data'
DATA_SRC='../data/raw/5k_subset'
# add src scripts to utilize
import sys
sys.path.append('../src')
sys.path.append('../src/data/')
sys.path.append('../src/models/')
sys.path.append('../src/features/')
sys.path.append('../src/visualization/')
# will reload any library
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load libraries

In [34]:
import os
import json
# utility
from tqdm import tqdm_notebook as tqdm
# scientific python
import numpy as np
import pandas as pd
# nice printing
from pprint import pprint
# for plotting
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Demo of process_mpd

Located in data/data_utils.py, **process_mpd** will take a directory path, a function to operate on a playlist and a object to store results (could be a dataframe or a dict for example). 

Optional is a *max_n* (max number of slices to operate on) and use *rand=False* for random or sorted files.


## counting the number of playlist

In [37]:
from data_utils import process_mpd

def count_playlist(playlist,results):
    results['count']=results['count']+1
    return

In [ ]:
results={}
results['count']=0
process_mpd(DATA_SRC,count_playlist,results,max_n=10,rand=True)
pprint(results)

## get unique tracks in all playlist

In [9]:
from data_utils import process_mpd, normalize_name

def unique_trackids(playlist,results):
    pid = playlist['pid']
    for track in playlist['tracks']:
        results.append({'title':track['track_name'],'artist':track['artist_name'],'uri':track['track_uri'],'pid':pid})
    return

In [10]:
results=[]
process_mpd(DATA_SRC,unique_trackids,results,max_n=5)
print(len(results))
#pd.DataFrame({'track_uri':list(results)}).to_csv('../data/interim/5k_track_uri.csv',index=False,compression='gzip')

HBox(children=(IntProgress(value=0, max=5), HTML(value='')))

335094


In [16]:
df = pd.DataFrame(results)
print(df.shape)
df = df.drop_duplicates('uri')
print(df.shape)
df.to_csv('../data/interim/5k_track_uri.csv',index=False,compression='gzip')

(335094, 4)
(107781, 4)


## create random subset of 1% of all playlists

In [37]:
from data_utils import process_mpd
import random

def coinflip(percent=0.001):
    return random.uniform(0, 1) < percent

def random_subset(playlist,results):
    if coinflip(1):
        results.append(playlist)
    return

In [ ]:
results=[]
process_mpd(DATA_SRC,random_subset,results)
print(len(results))
pprint(results[-1])

 86%|████████▋ | 863/1000 [06:15<00:59,  2.30it/s]

In [27]:
a_dict={"info": {
        "generated_on": "2017-12-03 08:41:42.057563", 
        "slice": "random subset of 0.1% of playlist", 
        "version": "v1"
    }, "playlists": results}
with open('../data/intetim/random_01_percent.json','w') as afile:
    afile.write(json.dumps(a_dict, indent=4))